In [1]:
import numpy as np
import pandas as pd
import json
import os

## Read all files

In [2]:
# Combine these two tables 
ori = pd.read_csv('fromjson_photos.csv')
photos = pd.read_csv('fromcsv_photos.csv')

# Combine these two
toy = pd.read_csv('fromcsv_toy.csv')
toy_coord = pd.read_csv('fromjson_toyCoord.csv')

# Combine these two
mon = pd.read_csv('fromcsv_monkey.csv')
mon_coord = pd.read_csv('fromjson_mCoord.csv')

# Finalize PHOTOS Table

In [3]:
photos['orientation'] = ori['orientation']
photos['contents'] = 'monkey'
display(photos.head(10))
photos.to_csv('photos_FINAL.csv', index=False)
print("PHOTOS TABLE IS SAVED!")

,jpg_id,file_name,sd_card,date_taken,monkey_group,orientation,contents
0,0,20220308_SD02_2923.JPG,SD02,2022-03-08,Stranger Things,portait,monkey
1,1,20220308_SD02_2924.JPG,SD02,2022-03-08,Stranger Things,portait,monkey
2,2,20220308_SD02_2946.JPG,SD02,2022-03-08,Zombies,portait,monkey
3,3,20220308_SD02_2954.JPG,SD02,2022-03-08,Zombies,landscape,monkey
4,4,20220308_SD02_2955.JPG,SD02,2022-03-08,Zombies,landscape,monkey
5,5,20220308_SD02_2956.JPG,SD02,2022-03-08,Zombies,portait,monkey
6,6,20220308_SD02_2958.JPG,SD02,2022-03-08,Zombies,portait,monkey
7,7,20220308_SD02_2959.JPG,SD02,2022-03-08,Zombies,portait,monkey
8,8,20220308_SD02_2960.JPG,SD02,2022-03-08,Zombies,portait,monkey
9,9,20220308_SD02_2961.JPG,SD02,2022-03-08,Zombies,portait,monkey


PHOTOS TABLE IS SAVED!


# Finalize TOY Table


In [4]:
toy_coord.head()

,jpg_id,toy,coordinates
0,34,Unknown,"(0.0, 0.895628, 0.16083, 1.0)"
1,38,Unknown,"(0.7477809999999999, 0.818182, 1.0, 0.97861000..."
2,60,DentalStar,"(0.6565219999999999, 0.727354, 0.947826, 0.825..."
3,80,Dumbbell,"(0.321212, 0.748602, 1.0, 1.0)"
4,163,Grenade,"(0.40405599999999997, 0.365112, 0.528861, 0.61..."


In [5]:
toy.head()

,jpg_id,toy
0,38,Unknown
1,60,DentalStar
2,79,Dumbbell
3,163,Grenade
4,163,Grenade


In [6]:
# Merge two dataframes
toy_merged = pd.merge(toy, toy_coord, on = ['jpg_id','toy'], how = 'outer')
toy_merged = toy_merged.sort_values('jpg_id', ignore_index = True)
toy_merged = toy_merged.reset_index().rename(columns={'index': 'toy_id'})
display(toy_merged.head())

# Check how many NaNs are there
toy_merged['coordinates'].isna().sum()

,toy_id,jpg_id,toy,coordinates
0,0,34,Unknown,"(0.0, 0.895628, 0.16083, 1.0)"
1,1,38,Unknown,"(0.7477809999999999, 0.818182, 1.0, 0.97861000..."
2,2,60,DentalStar,"(0.6565219999999999, 0.727354, 0.947826, 0.825..."
3,3,79,Dumbbell,NaN
4,4,80,Dumbbell,"(0.321212, 0.748602, 1.0, 1.0)"


6

In [7]:
toy_merged[toy_merged.coordinates.isnull()]

,toy_id,jpg_id,toy,coordinates
3,3,79,Dumbbell,NaN
112,112,938,Unknown,NaN
294,294,1787,Kong,NaN
435,435,2258,Grenade,NaN
469,469,2478,MegalastBall,NaN
513,513,2896,Unknown,NaN


In [8]:
# Fix the wrong ones -- i checked the actual photos and these toys are not there
toy_merged = toy_merged.drop(3) # jpg 79
toy_merged = toy_merged.drop(112) # jpg 938
toy_merged = toy_merged.drop(294) # jpg 1787
toy_merged = toy_merged.drop(435) # jpg 2258
toy_merged = toy_merged.drop(469) # jpg 2478
toy_merged = toy_merged.drop(513) # jpg 2896

In [9]:
toy_merged[toy_merged.coordinates.isnull()]

,toy_id,jpg_id,toy,coordinates


In [10]:
# Reset toy_id
toy_merged = toy_merged.drop('toy_id', axis=1)
toy_merged = toy_merged.sort_values('jpg_id', ignore_index = True)
toy_merged = toy_merged.reset_index(drop=True)
toy_merged = toy_merged.reset_index().rename(columns={'index': 'toy_id'})
toy_merged.tail(5)

,toy_id,jpg_id,toy,coordinates
1039,1039,6653,ChallengerBall,"(0.5826089999999999, 0.0, 1.0, 0.3202100000000..."
1040,1040,6654,ChallengerBall,"(0.6869569999999999, 0.001449, 1.0, 0.22458199..."
1041,1041,6695,Kong,"(0.8576849999999999, 0.8910309999999999, 0.944..."
1042,1042,6696,Kong,"(0.8842500000000001, 0.928039, 0.963947, 1.0)"
1043,1043,6697,Kong,"(0.899431, 0.939426, 0.9658439999999999, 1.0)"


In [11]:
# SAVE 
toy_merged.to_csv('toys_FINAL.csv', index=False)
print("TOYS TABLE IS SAVED!")

TOYS TABLE IS SAVED!


# Finalize MONKEY Table AND COORDINATES Table

In [12]:
display(mon_coord.head())
display(mon.head())

,jpg_id,coordinates,monkey_name,body_part
0,0,"(0.36186799999999997, 0.0028209999999999997, 0...",DF2I,Face
1,0,"(0.0, 0.0, 0.954399, 1.0)",DF2I,Body
2,1,"(0.34282199999999996, 0.0, 0.8845649999999999,...",DF2I,Face
3,1,"(0.0, 0.0028209999999999997, 0.975560999999999...",DF2I,Body
4,2,"(0.002116, 0.0, 0.387262, 0.7334269999999999)",7124,Body


,jpg_id,monkey_name,unknown_guess
0,0,DF2I,NaN
1,1,DF2I,NaN
2,2,7124,NaN
3,2,110E,NaN
4,2,151J,NaN


In [13]:
# Merge two dataframes
monkey_merged = pd.merge(mon, mon_coord, on = ['jpg_id', 'monkey_name'], how = 'outer')
monkey_merged = monkey_merged.sort_values(by=['jpg_id','monkey_name'])
monkey_merged.sample(n=10)
monkey_merged.head()

,jpg_id,monkey_name,unknown_guess,coordinates,body_part
0,0,DF2I,NaN,"(0.36186799999999997, 0.0028209999999999997, 0...",Face
1,0,DF2I,NaN,"(0.0, 0.0, 0.954399, 1.0)",Body
2,1,DF2I,NaN,"(0.34282199999999996, 0.0, 0.8845649999999999,...",Face
3,1,DF2I,NaN,"(0.0, 0.0028209999999999997, 0.975560999999999...",Body
6,2,110E,NaN,"(0.101577, 0.09591000000000001, 0.971329, 1.0)",Body


In [14]:
# Pivot to separate out body_parts
pivoted = monkey_merged.pivot(index = ['jpg_id','monkey_name'], columns='body_part', values='coordinates')
pivoted = pivoted.sort_values(by=['jpg_id','monkey_name'])
pivoted = pivoted.reset_index() # reset index to avoid multi-indexing
pivoted.head(7)

body_part,jpg_id,monkey_name,NaN,Body,Face
0,0,DF2I,NaN,"(0.0, 0.0, 0.954399, 1.0)","(0.36186799999999997, 0.0028209999999999997, 0..."
1,1,DF2I,NaN,"(0.0, 0.0028209999999999997, 0.975560999999999...","(0.34282199999999996, 0.0, 0.8845649999999999,..."
2,2,110E,NaN,"(0.101577, 0.09591000000000001, 0.971329, 1.0)","(0.118506, 0.094499, 0.5100009999999999, 0.389..."
3,2,151J,NaN,"(0.126971, 0.35543, 0.8591709999999999, 0.943583)","(0.152365, 0.35543, 0.5100009999999999, 0.558533)"
4,2,7124,NaN,"(0.002116, 0.0, 0.387262, 0.7334269999999999)","(0.0, 0.0, 0.378797, 0.31029599999999996)"
5,3,110E,NaN,"(0.40347099999999997, 0.195258, 0.936009, 1.0)","(0.408894, 0.20014, 0.652928, 0.6150639999999999)"
6,3,151J,NaN,"(0.41214799999999996, 0.584148, 0.854664, 1.0)","(0.42407799999999995, 0.587402, 0.616051999999..."


In [15]:
# Check if there are duplicate monkey_name within the same photo
max_jpgid = pivoted['jpg_id'].max()
min_jpgid = pivoted['jpg_id'].min()

count = 0
for i in range(min_jpgid, max_jpgid+1):
    temp = pivoted[pivoted['jpg_id']==i]
    if temp['monkey_name'].duplicated().any():
        print(temp)
        count += 1
        
print("There are", count, "photos with duplicate monkey_name")

There are 0 photos with duplicate monkey_name


In [16]:
# Give monkey_id to pivot table
pivoted = pivoted.reset_index(drop=True)
pivoted = pivoted.reset_index().rename(columns={'index': 'monkey_id'})
pivoted

body_part,monkey_id,jpg_id,monkey_name,NaN,Body,Face
0,0,0,DF2I,NaN,"(0.0, 0.0, 0.954399, 1.0)","(0.36186799999999997, 0.0028209999999999997, 0..."
1,1,1,DF2I,NaN,"(0.0, 0.0028209999999999997, 0.975560999999999...","(0.34282199999999996, 0.0, 0.8845649999999999,..."
2,2,2,110E,NaN,"(0.101577, 0.09591000000000001, 0.971329, 1.0)","(0.118506, 0.094499, 0.5100009999999999, 0.389..."
3,3,2,151J,NaN,"(0.126971, 0.35543, 0.8591709999999999, 0.943583)","(0.152365, 0.35543, 0.5100009999999999, 0.558533)"
4,4,2,7124,NaN,"(0.002116, 0.0, 0.387262, 0.7334269999999999)","(0.0, 0.0, 0.378797, 0.31029599999999996)"
...,...,...,...,...,...,...
11157,11157,6746,G942,NaN,"(0.091253, 0.011314999999999999, 0.88282299999...","(0.101864, 0.132956, 0.297104, 0.37058)"
11158,11158,6747,G942,NaN,"(0.140063, 0.008487, 0.9401219999999999, 1.0)","(0.144308, 0.132956, 0.324692, 0.367751)"
11159,11159,6748,G942,NaN,"(0.0, 0.173975, 0.8955559999999999, 0.876945)","(0.38199099999999997, 0.175389, 0.672728, 0.38..."
11160,11160,6749,G942,NaN,"(0.0021219999999999998, 0.168317, 1.0, 0.929279)","(0.581475, 0.164074, 0.846746, 0.424328)"


In [17]:
# Generate MONKEY table and COORDINATES table
monkey_table = pivoted[['monkey_id','jpg_id','monkey_name']].copy()
coordinate_table = pivoted[['monkey_id','Body','Face']].copy()

In [18]:
# Validate that monkey_merged and pivoted have the same jpg_id, monkey_name
val_1 = monkey_merged[['jpg_id', 'monkey_name']].copy().drop_duplicates()
val_2 = pivoted[['jpg_id','monkey_name']].copy()

print((val_2['jpg_id'].values == val_1['jpg_id'].values).all())
print((val_2['monkey_name'].values == val_1['monkey_name'].values).all())

True
True


In [19]:
# SAVE Monkey Table
monkey_table.to_csv('monkey_FINAL.csv', index=False)
print("MONKEY TABLE SAVED!!!")

MONKEY TABLE SAVED!!!


In [20]:
# SAVE Coordinate Table
coordinate_table.to_csv('coordinate_FINAL.csv', index=False)
print("COORDINATE TABLE SAVED!!!")

COORDINATE TABLE SAVED!!!


# Finalize UNKNOWN Table

In [21]:
# Extract components from monkey_merged
unknown = monkey_merged[['jpg_id', 'monkey_name','unknown_guess']].copy().drop_duplicates()

# Give monkey_id
unknown = unknown.reset_index(drop=True)
unknown = unknown.reset_index().rename(columns={'index': 'monkey_id'})
unknown

,monkey_id,jpg_id,monkey_name,unknown_guess
0,0,0,DF2I,NaN
1,1,1,DF2I,NaN
2,2,2,110E,NaN
3,3,2,151J,NaN
4,4,2,7124,NaN
...,...,...,...,...
11157,11157,6746,G942,NaN
11158,11158,6747,G942,NaN
11159,11159,6748,G942,NaN
11160,11160,6749,G942,NaN


In [22]:
# Extract Unknowns only
unknown_table = unknown[unknown['monkey_name'].str.contains('unknown', case = False)]
unknown_table.head()

,monkey_id,jpg_id,monkey_name,unknown_guess
67,67,46,Unknown1,NaN
73,73,50,Unknown1,NN
90,90,60,Unknown1,"I, NN"
91,91,60,Unknown2,"I, NN"
96,96,64,Unknown1,"Juvenile, NN"


In [23]:
# Take care of multi-valued field in 'unknown_guess'
for index, row in unknown_table.iterrows():
    if 'Unknown1' in row['monkey_name'] and ',' in str(row['unknown_guess']):
        parts = row['unknown_guess'].split(',')
        unknown1 = parts[0].strip()
        unknown_table.loc[index,'unknown_guess'] = str(unknown1)
        
    elif 'Unknown2' in row['monkey_name'] and ',' in str(row['unknown_guess']):
        parts = row['unknown_guess'].split(',')
        unknown2 = parts[1].strip()
        unknown_table.loc[index,'unknown_guess'] = unknown2

unknown_table.head(10)

,monkey_id,jpg_id,monkey_name,unknown_guess
67,67,46,Unknown1,NaN
73,73,50,Unknown1,NN
90,90,60,Unknown1,I
91,91,60,Unknown2,NN
96,96,64,Unknown1,Juvenile
97,97,64,Unknown2,NN
99,99,65,Unknown1,Juvenile
105,105,69,Unknown1,Adult Female
107,107,70,Unknown1,Adult Female
111,111,73,Unknown1,J


In [24]:
unknown_table = unknown_table.drop(['jpg_id','monkey_name'], axis = 1)
unknown_table.head()

,monkey_id,unknown_guess
67,67,NaN
73,73,NN
90,90,I
91,91,NN
96,96,Juvenile


In [25]:
unknown_table.to_csv('unknown_FINAL.csv', index=False)
print("UNKNOWN TABLE SAVED!!!")

UNKNOWN TABLE SAVED!!!
